In [1]:

import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
datos = pd.read_csv("Icfeslimpio.csv")
datos.shape

C:\Users\oscar\AppData\Local\Temp\ipykernel_14636\1681162076.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("Icfeslimpio.csv")


(115835, 38)

In [3]:
# Eliminar identificador comun
datos.drop("estu_consecutivo", axis=1, inplace=True)

In [4]:
datos.nunique()

estu_tipodocumento                 11
cole_area_ubicacion                 2
cole_bilingue                       2
cole_caracter                       4
cole_cod_dane_establecimiento     216
cole_cod_dane_sede                222
cole_cod_mcpio_ubicacion          144
cole_genero                         3
cole_jornada                        6
cole_naturaleza                     2
estu_cod_depto_presentacion        25
estu_cod_mcpio_presentacion       106
estu_cod_reside_depto              27
estu_cod_reside_mcpio             200
estu_estadoinvestigacion            4
estu_genero                         2
estu_nacionalidad                  15
estu_privado_libertad               1
fami_cuartoshogar                   1
fami_educacionmadre                12
fami_educacionpadre                12
fami_estratovivienda                7
fami_personashogar                  5
fami_tieneautomovil                 2
fami_tienecomputador                2
fami_tieneinternet                  2
fami_tienela

In [5]:
# se dummifica las variables con menos de 20 categorias
lista = []
for column in datos.columns.to_list():
    if datos.nunique()[column] < 20 and column != "año":
        lista.append(column)

In [6]:
datos = pd.get_dummies(datos,columns=lista, drop_first=False)

In [7]:
datos["año_nacimiento"] = datos["estu_fechanacimiento_dt"].astype(str).str[:4].astype(int)
datos.drop("estu_fechanacimiento_dt", axis=1, inplace=True)

In [8]:
# se utiliza target encoding para las variables de más de 20 categorias
promedio_global = datos['punt_global'].mean()
dics = {}
for column in datos.columns.to_list():
    if datos.nunique()[column] > 20 and "punt" not in column and "año" not in column:
        # Calculate mean house price for each city
        mean_encoded = datos.groupby(column)['punt_global'].mean()
        smooth_encoded = (mean_encoded * datos[column].value_counts() + promedio_global) / (datos[column].value_counts() + 1)
        datos[f'{column}_encoded'] = datos[column].map(smooth_encoded)
        coded = pd.DataFrame()
        coded[f'{column}_encoded']= datos[f'{column}_encoded']
        coded[column] = datos[column]
        dics[column] = coded
        datos.drop(column, axis=1, inplace = True)

In [9]:

datos.head()

,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global,año,estu_tipodocumento_CC,estu_tipodocumento_CE,estu_tipodocumento_CR,...,semestre_2,semestre_4,año_nacimiento,cole_cod_dane_establecimiento_encoded,cole_cod_dane_sede_encoded,cole_cod_mcpio_ubicacion_encoded,estu_cod_depto_presentacion_encoded,estu_cod_mcpio_presentacion_encoded,estu_cod_reside_depto_encoded,estu_cod_reside_mcpio_encoded
0,42.0,58.0,51.0,52.0,59.0,270.0,2019,False,False,False,...,False,True,2002,275.551974,275.554202,273.995615,253.908532,270.141257,253.939348,273.830665
1,60.0,64.0,60.0,59.0,67.0,312.0,2016,False,False,False,...,True,False,1999,276.955395,276.955395,265.045932,253.908532,263.838632,253.939348,265.826057
2,55.0,56.0,54.0,54.0,60.0,280.0,2017,False,False,False,...,True,False,2000,275.551974,275.554202,273.995615,253.908532,270.141257,253.939348,273.830665
3,50.0,45.0,40.0,38.0,40.0,206.0,2016,True,False,False,...,True,False,1997,199.298165,199.298165,199.298165,253.908532,208.304425,253.939348,200.079270
4,54.0,51.0,47.0,46.0,54.0,249.0,2017,False,False,False,...,True,False,1999,265.398074,265.398074,265.398074,253.908532,270.141257,253.939348,262.885582


In [10]:
datos.to_csv("datos_modelo.csv", index=False)

In [11]:
for key in dics:
    dics[key].to_csv(f"dic_{key}.csv", index=False)